In [ ]:
!pip install h5py
!pip install typing-extensions
!pip install wheel

In [32]:
!pip -q install chromadb openai tiktoken pypdf
!pip install -U langchain-community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00


In [10]:
!pip show chromadb

Name: chromadb
Version: 0.5.18
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, httpx, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, rich, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 


In [ ]:
import os
openai.api_key = os.getenv('OPENAI_API_KEY')

In [33]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain import embeddings

In [21]:
loader = PyPDFLoader(
    "./spaul/Deep-Learning-with-PyTorch.pdf",
)
document = loader.load()

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)
text = text_splitter.split_documents(document)

In [27]:
text[1]

Document(metadata={'source': './spaul/Deep-Learning-with-PyTorch.pdf', 'page': 1}, page_content='Deep Learning with PyTorch\nEssential Excerpts\nEli Stevens and Luca Antiga\n \nManning Author Picks\n    Copyright 2019 Manning Publications\nTo pre-order or learn more about this book go to  www.manning.com')

Create database

In [35]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=text,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [44]:
# Persist the db to disk
vectordb.persist()
vectordb = None

In [42]:
# Now the persisted database can be loaded from disk
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

<ipython-input-42-8621d7e0c64e>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [46]:
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents("What is natural language processing?")

In [ ]:
retriever.search_type

Extract the OpenAI API response using LangChain

In [ ]:
from langchain.chains import RetrievalQA
llm=OpenAI()
llm

In [ ]:
# Create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
query = "What is natural language processing?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Delete the database

In [43]:
!zip -r db.zip ./db

# To cleanup, delete the collection
vectordb.delete_collection()
vectordb.persist()

# Delete the directory
!rm -rf db/

updating: db/ (stored 0%)
updating: db/chroma.sqlite3 (deflated 97%)


<ipython-input-43-872275e7124e>:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


Reload the database

In [ ]:
!unzip db.zip